In [1]:
import sagemaker
sess = sagemaker.Session()

In [2]:
%%sh
wget -N https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip
unzip -o bank-additional.zip

Archive:  bank-additional.zip
   creating: bank-additional/
  inflating: bank-additional/bank-additional-names.txt  
  inflating: bank-additional/bank-additional.csv  
  inflating: bank-additional/bank-additional-full.csv  


--2021-07-31 13:34:42--  https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip
Resolving sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com (sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com)... 52.92.162.194
Connecting to sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com (sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com)|52.92.162.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 432828 (423K) [application/zip]
Saving to: ‘bank-additional.zip’

     0K .......... .......... .......... .......... .......... 11%  164K 2s
    50K .......... .......... .......... .......... .......... 23%  103M 1s
   100K .......... .......... .......... .......... .......... 35%  320K 1s
   150K .......... .......... .......... .......... .......... 47% 35.8M 1s
   200K .......... .......... .......... .......... .......... 59% 1.18M 0s
   250K .......... .......... .......... .....

In [3]:
bucket = sess.default_bucket()                     
prefix = 'sagemaker/DEMO-automl-dm'
s3_input_data = sess.upload_data(path="./bank-additional/bank-additional-full.csv", key_prefix=prefix+'input')

In [4]:
! head bank-additional/bank-additional-full.csv

age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
45,services,married,basic.9y,unknown,no,no,telephone,may,mon,198,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
59,admin.,married,professional.course,no,no,no,telephone,may,mon,139,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no

In [5]:
from sagemaker.automl.automl import AutoML
auto_ml_job = AutoML(
    role = sagemaker.get_execution_role(),                                          
    sagemaker_session = sess,                             
    target_attribute_name = 'y',                             
    output_path = 's3://{}/{}/output'.format(bucket,prefix),
    max_runtime_per_training_job_in_seconds = 600,
    max_candidates = 250,
    total_job_runtime_in_seconds = 3600
)

In [6]:
auto_ml_job.fit(inputs=s3_input_data, logs=False, wait=False)

In [7]:
from time import sleep

job = auto_ml_job.describe_auto_ml_job()
job_status = job['AutoMLJobStatus']
job_sec_status = job['AutoMLJobSecondaryStatus']
if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('AnalyzingData'):
        sleep(30)
        job = auto_ml_job.describe_auto_ml_job()
        job_status = job['AutoMLJobStatus']
        job_sec_status = job['AutoMLJobSecondaryStatus']
        print (job_status, job_sec_status)

InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress FeatureEngineering


In [8]:
job = auto_ml_job.describe_auto_ml_job()
#print(job)
job_candidate_notebook = job['AutoMLJobArtifacts']['CandidateDefinitionNotebookLocation']
job_data_notebook = job['AutoMLJobArtifacts']['DataExplorationNotebookLocation']
print(job_candidate_notebook)
print(job_data_notebook)

s3://sagemaker-eu-central-1-839447162432/sagemaker/DEMO-automl-dm/output/automl-2021-07-31-13-35-27-627/sagemaker-automl-candidates/automl-2021-07-31-13-35-27-627-pr-1-117a186e23fe4b2eb1a9babdfbe/notebooks/SageMakerAutopilotCandidateDefinitionNotebook.ipynb
s3://sagemaker-eu-central-1-839447162432/sagemaker/DEMO-automl-dm/output/automl-2021-07-31-13-35-27-627/sagemaker-automl-candidates/automl-2021-07-31-13-35-27-627-pr-1-117a186e23fe4b2eb1a9babdfbe/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb


In [9]:
%%sh -s $job_candidate_notebook $job_data_notebook
aws s3 cp $1 .
aws s3 cp $2 .

download: s3://sagemaker-eu-central-1-839447162432/sagemaker/DEMO-automl-dm/output/automl-2021-07-31-13-35-27-627/sagemaker-automl-candidates/automl-2021-07-31-13-35-27-627-pr-1-117a186e23fe4b2eb1a9babdfbe/notebooks/SageMakerAutopilotCandidateDefinitionNotebook.ipynb to ./SageMakerAutopilotCandidateDefinitionNotebook.ipynb
download: s3://sagemaker-eu-central-1-839447162432/sagemaker/DEMO-automl-dm/output/automl-2021-07-31-13-35-27-627/sagemaker-automl-candidates/automl-2021-07-31-13-35-27-627-pr-1-117a186e23fe4b2eb1a9babdfbe/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb to ./SageMakerAutopilotDataExplorationNotebook.ipynb


In [10]:
job = auto_ml_job.describe_auto_ml_job()
job_status = job['AutoMLJobStatus']
job_sec_status = job['AutoMLJobSecondaryStatus']
if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('FeatureEngineering'):
        sleep(30)
        job = auto_ml_job.describe_auto_ml_job()
        job_status = job['AutoMLJobStatus']
        job_sec_status = job['AutoMLJobSecondaryStatus']
        print (job_status, job_sec_status)

InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress ModelTuning


In [11]:
import pandas as pd
from sagemaker.analytics import ExperimentAnalytics

exp = ExperimentAnalytics(
    sagemaker_session=sess,
    experiment_name=job['AutoMLJobName'] + '-aws-auto-ml-job'
)
df = exp.dataframe()
print("Number of jobs: ", len(df))
if (len(df) !=0):
    df = pd.concat([df['ObjectiveMetric - Max'], df.drop(['ObjectiveMetric - Max'], axis=1)], axis=1)
    df.sort_values('ObjectiveMetric - Max', ascending=0)[:5]

Number of jobs:  220


In [12]:
job = auto_ml_job.describe_auto_ml_job()
job_status = job['AutoMLJobStatus']
job_sec_status = job['AutoMLJobSecondaryStatus']
if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('ModelTuning'):
        sleep(30)
        job = auto_ml_job.describe_auto_ml_job()
        job_status = job['AutoMLJobStatus']
        job_sec_status = job['AutoMLJobSecondaryStatus']
        print (job_status, job_sec_status)

In [13]:
job_best_candidate = auto_ml_job.best_candidate()
print(job_best_candidate['CandidateName'])
print(job_best_candidate['FinalAutoMLJobObjectiveMetric'])

automl-2021-07-31-13-35-27-627od-171-529d2657
{'MetricName': 'validation:f1_binary', 'Value': 0.655489981174469}


In [14]:
import boto3
job_outputs_prefix = '{}/output/{}'.format(prefix, job['AutoMLJobName'])
s3_bucket = boto3.resource('s3').Bucket(bucket)
s3_bucket.objects.filter(Prefix=job_outputs_prefix).delete()

[{'ResponseMetadata': {'RequestId': 'GMMJ7NER3KF71HPT',
   'HostId': 'YgR0ndZUCOvl+RUvE4IEPaeRugy5rn61/XtjecIXxvXGeX3GRkKEEfm1JeJrOJfhY/dZfAoH5ow=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'YgR0ndZUCOvl+RUvE4IEPaeRugy5rn61/XtjecIXxvXGeX3GRkKEEfm1JeJrOJfhY/dZfAoH5ow=',
    'x-amz-request-id': 'GMMJ7NER3KF71HPT',
    'date': 'Sat, 31 Jul 2021 14:38:04 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'sagemaker/DEMO-automl-dm/output/automl-2021-07-31-13-35-27-627/transformed-data/dpp2/rpb/train/chunk_39.csv.out'},
   {'Key': 'sagemaker/DEMO-automl-dm/output/automl-2021-07-31-13-35-27-627/transformed-data/dpp5/rpb/train/chunk_46.csv.out'},
   {'Key': 'sagemaker/DEMO-automl-dm/output/automl-2021-07-31-13-35-27-627/transformed-data/dpp6/csv/train/chunk_53.csv.out'},
   {'Key': 'sagemaker/DEMO-automl-dm/output/automl-2021-07-31-13-35-27-627/sag